# LLM을 프로그램 코드 생성에 활용하는 것 연습

LCEL(LangChain Expression Langague) 문서 중 Cookbook의 Code writing 부분을 참조하였음
&rarr; https://python.langchain.com/docs/expression_language/cookbook/code_writing

In [1]:
%pip install -q langchain-experimental 

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    ChatPromptTemplate,
)
from langchain_experimental.utilities import PythonREPL
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from IPython.display import display, Markdown, Code, Pretty


In [3]:
def _sanitize_output(text: str):
    try:
        _, after = text.split("```python")
    except:
        return ''
    return after.split("```")[0]

In [4]:
template = """
Write some python code to solve the user's problem. 
Use numpy.mean and numpy.std to calculate average and standard deviation of an array. 
Avoid importing descriptive_statistics from scipy.stats.

Return only python code in Markdown format, e.g.:

```python
....
```
"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

# model = ChatOpenAI()
# model = ChatOllama(model="stable-code")
# model = ChatOllama(model="dolphin-mistral")
model = ChatOllama(model="codellama")


In [5]:
# chain = prompt | model | StrOutputParser() | _sanitize_output | PythonREPL().run
chain = prompt | model | StrOutputParser() | _sanitize_output

In [6]:
# 가끔 아무 코드가 안 나올때가 있음. 강제로 10번 반복.
for i in range(10):
    code = chain.invoke({"input": "print out 10 random numbers ranged from 10 to 20"})
    # code = chain.invoke({"input": "what is 2 plus 2"})
    # code = chain.invoke({"input": "print all possible combinations of items in [1, 2, 3]"})
    if len(code) > 0:
        try:
            exec(code)
        except:
            pass
        break

[3 2 5 5 8 8 7 0 3 5]


In [7]:
display(Code(code))

import numpy as np

# Generate 10 random numbers between 10 and 20
numbers = np.random.randint(10, size=10)

print(numbers)

In [8]:
# 좀 더 복잡한 것도 시켜보자.
for i in range(10):
    code = chain.invoke({"input": "generate 100 batches of 10 random numbers ranged from 10 to 20 and calculate sum of each batch and print average and standard deviation of sums"})
    if len(code) > 0:
        try:
            exec(code)
        except:
            pass
        break

Average: 146.01
Standard Deviation: 9.572350808448256


In [9]:
code

'\nimport numpy as np\n\n# generate 100 batches of 10 random numbers ranged from 10 to 20\nrandom_numbers = np.random.randint(10, 20, size=(100, 10))\n\n# calculate sum of each batch\nbatch_sums = random_numbers.sum(axis=1)\n\n# calculate average and standard deviation of sums\naverage = batch_sums.mean()\nstd_dev = batch_sums.std()\n\nprint("Average:", average)\nprint("Standard Deviation:", std_dev)\n'

In [10]:
display(Code(code))

import numpy as np

# generate 100 batches of 10 random numbers ranged from 10 to 20
random_numbers = np.random.randint(10, 20, size=(100, 10))

# calculate sum of each batch
batch_sums = random_numbers.sum(axis=1)

# calculate average and standard deviation of sums
average = batch_sums.mean()
std_dev = batch_sums.std()

print("Average:", average)
print("Standard Deviation:", std_dev)

Following examples at https://ollama.ai/library/codellama

In [11]:
prompt = ChatPromptTemplate.from_template("{input}")

model = ChatOllama(model="codellama")

In [12]:
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [13]:
code = chain.invoke({"input": "Write me a function that outputs the fibonacci sequence"})

In [14]:
code

'\nfunction fibonacci(n) {\n  if (n <= 1) return n;\n  return fibonacci(n-1) + fibonacci(n-2);\n}\n\n// Test case:\nconsole.log(fibonacci(5)); // should output 5\nconsole.log(fibonacci(8)); // should output 21\nconsole.log(fibonacci(13)); // should output 89\nconsole.log(fibonacci(20)); // should output 6765\n```'

In [16]:
display(Code(code))

function fibonacci(n) {
  if (n <= 1) return n;
  return fibonacci(n-1) + fibonacci(n-2);
}

// Test case:
console.log(fibonacci(5)); // should output 5
console.log(fibonacci(8)); // should output 21
console.log(fibonacci(13)); // should output 89
console.log(fibonacci(20)); // should output 6765
```

Instruct variation

In [17]:
template = """
You are an expert programmer that writes simple, concise code and explanations in Markdown format. 
"""
prompt = ChatPromptTemplate.from_messages([("system", template), ("human", "{input}")])

model = ChatOllama(model="codellama:7b-instruct")

In [18]:
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [19]:
code = chain.invoke({"input": "Write a python function to generate the nth fibonacci number."})

In [20]:
code

'\n```\ndef fibonacci(n):\n    if n <= 1:\n        return n\n    else:\n        return fibonacci(n-1) + fibonacci(n-2)\n```\n\nExplanation:\n\nThe `fibonacci` function takes an integer `n` as input and returns the `nth` Fibonacci number. The function uses a recursive approach, where it calls itself with smaller values of `n` until it reaches the base case. In the base case, when `n <= 1`, the function simply returns the value of `n`. Otherwise, it calculates the sum of the previous two Fibonacci numbers using the same formula and returns that result.\n\nFor example, if we call the function with `n = 5`, it will calculate the Fibonacci sequence as follows:\n```\nfibonacci(5) = fibonacci(4) + fibonacci(3)\n            = fibonacci(3) + fibonacci(2) + fibonacci(1)\n            = 3 + 2 + 1\n            = 6\n```\nSo, the value of `fibonacci(5)` is `6`.'

In [21]:
display(Markdown(code))


```
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)
```

Explanation:

The `fibonacci` function takes an integer `n` as input and returns the `nth` Fibonacci number. The function uses a recursive approach, where it calls itself with smaller values of `n` until it reaches the base case. In the base case, when `n <= 1`, the function simply returns the value of `n`. Otherwise, it calculates the sum of the previous two Fibonacci numbers using the same formula and returns that result.

For example, if we call the function with `n = 5`, it will calculate the Fibonacci sequence as follows:
```
fibonacci(5) = fibonacci(4) + fibonacci(3)
            = fibonacci(3) + fibonacci(2) + fibonacci(1)
            = 3 + 2 + 1
            = 6
```
So, the value of `fibonacci(5)` is `6`.

Fill in the middle or infill

In [22]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("<PRE> {prefix} <SUF>{suffix} <MID>")
llm = Ollama(base_url="http://localhost:11434", model="codellama:7b-code")

In [23]:
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [24]:
code = chain.invoke({"prefix": "def compute_gcd(x, y):", "suffix": "return result"})

In [25]:
display(Code(code))

if x == 0:
    return y
  elif y == 0:
    return x
  else:
    m = min(x, y)
    while m > 0:
      if (m % x == 0 and m % y == 0):
        return m
      else:
        m -= 1

def compute_lcm(x, y):
  result = x * y / compute_gcd(x, y)
   <EOT>

Code review

In [26]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{input}")
llm = Ollama(base_url="http://localhost:11434", model="codellama")

In [27]:
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [28]:
code = chain.invoke({"input":"""
Where is the bug in this code?

def fib(n):
    if n <= 0:
        return n
    else:
        return fib(n-1) + fib(n-2)
"""})

In [29]:
display(Markdown(code))


The bug in this code is that it will cause a recursion error for values of `n` greater than or equal to 47, as the function calls itself with arguments that are one less than the previous call. This will eventually cause the stack to overflow, causing a recursion error.

To fix the bug, you can modify the function to use a loop instead of recursive calls, like this:
```
def fib(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a+b
    return a
```
This will work correctly for any value of `n`.

Writing tests

In [30]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{input}")
llm = Ollama(base_url="http://localhost:11434", model="codellama")

In [31]:
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [32]:
code = chain.invoke({"input":"""
write a unit test for this function: 

def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)
"""})


In [33]:
display(Markdown(code))

  Here is an example of a unit test for the `fibonacci` function using the built-in Python unittest module:
```
import unittest

class TestFibonacci(unittest.TestCase):
    def test_fibonacci(self):
        self.assertEqual(fibonacci(1), 1)
        self.assertEqual(fibonacci(2), 1)
        self.assertEqual(fibonacci(3), 2)
        self.assertEqual(fibonacci(4), 3)
        self.assertEqual(fibonacci(5), 5)
        self.assertEqual(fibonacci(6), 8)
        self.assertEqual(fibonacci(7), 13)
        self.assertEqual(fibonacci(8), 21)
        self.assertEqual(fibonacci(9), 34)
        self.assertEqual(fibonacci(10), 55)

if __name__ == '__main__':
    unittest.main()
```
This test case will check that the `fibonacci` function returns the correct values for the first 10 Fibonacci numbers.

You can also use a more generic approach to test the function, by generating random input and checking if the output is correct. Here is an example of such a test:
```
import unittest
import random

class TestFibonacci(unittest.TestCase):
    def test_fibonacci(self):
        for i in range(100):
            n = random.randint(1, 100)
            self.assertEqual(fibonacci(n), fibonacci_naive(n))

if __name__ == '__main__':
    unittest.main()
```
This test case will generate a random number between 1 and 100, pass it to the `fibonacci` function, and then compare the result with the result of calling the naive implementation of the Fibonacci sequence on the same input. This will help you to ensure that your recursive implementation is correct and returns the same results as the naive implementation for a large number of random inputs.

Code completion from comments

In [34]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{input}")
llm = Ollama(base_url="http://localhost:11434", model="codellama:7b-python")


In [35]:
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [36]:
code = chain.invoke({"input": "# A simple python function to remove whitespace from a string:"})

In [37]:
display(Code(code))

zef.function(
	name="remove_whitespace",
	purpose=Purpose.utility,
	inputs={"str": "The input string."},
	outputs={"newstring": "The new string without whitespace"},
	description="A simple function to remove whitespace from a string."
)
def remove_whitespace(str):
	return str.replace(" ","")

Cody 흉내내기 (1) 코멘트를 넣으면 자동으로 코드 채워주기 

In [38]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{input}")
llm = Ollama(base_url="http://localhost:11434", model="codellama:7b-python",
    temperature=0.2,
    top_k=-1,
    top_p=-1,
    stop=["\n","// Path:","\u001e","\u001c","<EOT>","; ",";\t"],
)

In [39]:
my_inputs = [
    "# open a text file \"a.txt\"\n",
    "# write hello to the file\n",
    "# close the file\n",
]

history = ""

In [40]:
for input in my_inputs:
    history += input
    retval = llm.invoke(history)
    history += f"{str(retval)}\n"
    print(history, end="")
    print("------------------------------------")


# open a text file "a.txt"
file = open("a.txt", "w")
------------------------------------
# open a text file "a.txt"
file = open("a.txt", "w")
# write hello to the file
file.write("hello\n")
------------------------------------
# open a text file "a.txt"
file = open("a.txt", "w")
# write hello to the file
file.write("hello\n")
# close the file
file.close()
------------------------------------


Cody 흉내내기 (2) 코드의 일부를 작성하면 자동으로 코드 채워주기 


In [41]:
llm = Ollama(base_url="http://localhost:11434", model="codellama:7b-python",
    temperature=0.2,
    top_k=-1,
    top_p=-1,
    stop=["\n","// Path:","\u001e","\u001c","<EOT>","; ",";\t"],
)

In [42]:
my_inputs = [
    "# sum 1 to 10 and print the sum\n"
    "for i in ",
    "\tsum = ",
    "print",
]

history = ""

In [43]:
for input in my_inputs:
    history += input
    retval = llm.invoke(history)
    history += f"{str(retval)}\n"
    print(history, end="")
    print("------------------------------------")


# sum 1 to 10 and print the sum
for i in  range(1,11):
------------------------------------
# sum 1 to 10 and print the sum
for i in  range(1,11):
	sum = 0
------------------------------------
# sum 1 to 10 and print the sum
for i in  range(1,11):
	sum = 0
print("Sum of 1 to 10 is:", sum)
------------------------------------
